https://dacon.io/competitions/official/235624/codeshare/1831?page=1&dtype=recent
## HAICon2020 산업제어시스템 보안위협 탐지 AI 경진대회

In [27]:
!pip install /home/ubuntu/coding/220906/eTaPR-1.12-py3-none-any.whl

Processing ./eTaPR-1.12-py3-none-any.whl
eTaPR is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


#### 1. 데이터 로드

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time

from keras.models import Model, Sequential, load_model
import keras
from keras import optimizers
from keras.layers import Input,Bidirectional ,LSTM, Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

from pathlib import Path
from datetime import timedelta
import dateutil
from tqdm.notebook import trange
from TaPR_pkg import etapr

In [57]:
TRAIN_DATASET = sorted([x for x in Path("/home/ubuntu/coding/220906/HAI 2.0/training").glob("*.csv")])
TRAIN_DATASET

[PosixPath('/home/ubuntu/coding/220906/HAI 2.0/training/train1.csv'),
 PosixPath('/home/ubuntu/coding/220906/HAI 2.0/training/train2.csv'),
 PosixPath('/home/ubuntu/coding/220906/HAI 2.0/training/train3.csv')]

In [58]:
TEST_DATASET = sorted([x for x in Path("/home/ubuntu/coding/220906/HAI 2.0/testing").glob("*.csv")])
TEST_DATASET

[PosixPath('/home/ubuntu/coding/220906/HAI 2.0/testing/test1.csv'),
 PosixPath('/home/ubuntu/coding/220906/HAI 2.0/testing/test2.csv'),
 PosixPath('/home/ubuntu/coding/220906/HAI 2.0/testing/test3.csv'),
 PosixPath('/home/ubuntu/coding/220906/HAI 2.0/testing/test4.csv')]

In [59]:
VALIDATION_DATASET = sorted([x for x in Path("/home/ubuntu/coding/220906/HAI 2.0/validation").glob("*.csv")])
VALIDATION_DATASET

[PosixPath('/home/ubuntu/coding/220906/HAI 2.0/validation/validation.csv')]

In [60]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

- Train dataset load
    - 설명: 정상적인 운영 상황에서 수집된 데이터(각 파일별로 시간 연속성을 가짐)
    - Column1 ('time'): 관측 시각
    - Column2, 3, …, 80 ('C01', 'C02', …, 'C79'): 상태 관측 데이터

In [61]:
TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
TRAIN_DF_RAW = TRAIN_DF_RAW[:30720]
TRAIN_DF_RAW

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,2020-07-11 00:00:00,395.19528,12,10,52.80456,-1.2648,-1.87531,779.59595,28.02645,10832.0,...,808.29620,0.0,1.36810,8.79882,35.43700,12.01782,305.03113,301.35992,33.6555,6.0951
1,2020-07-11 00:00:01,395.14420,12,10,52.78931,-1.3147,-1.88294,780.67328,28.02473,10984.0,...,819.16809,0.0,1.36810,8.78811,35.45227,12.01782,304.27161,297.43567,33.6555,5.9262
2,2020-07-11 00:00:02,395.14420,12,10,52.79694,-1.4032,-1.88294,780.06574,28.02817,11120.0,...,823.51697,0.0,1.36734,8.81787,35.45227,12.01782,303.89179,298.66534,33.6555,5.8101
3,2020-07-11 00:00:03,395.19528,12,10,52.79694,-1.6074,-1.88294,780.15265,28.02301,11256.0,...,823.95172,0.0,1.36734,8.87493,35.43700,12.01782,303.67474,298.06860,33.6555,5.7509
4,2020-07-11 00:00:04,395.34866,12,10,52.79694,-1.7811,-1.88294,781.83160,28.03595,11384.0,...,827.86560,0.0,1.36810,8.83838,35.45227,12.01782,303.22266,296.53137,33.6555,5.8547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30715,2020-07-11 08:31:55,379.65576,12,10,47.96753,-1.8486,65.95001,776.27032,28.02730,120.0,...,872.65772,0.0,1.42380,12.38278,35.60485,12.01782,356.02942,363.46210,32.0000,6.3387
30716,2020-07-11 08:31:56,380.16693,12,10,47.96753,-1.7524,64.14185,778.04956,28.02821,112.0,...,874.39728,0.0,1.42532,12.42272,35.62011,12.01019,356.89740,365.72266,32.0000,6.3871
30717,2020-07-11 08:31:57,380.62695,12,10,47.96753,-1.6188,62.25738,774.03912,28.02645,104.0,...,873.52753,0.0,1.42304,12.40054,35.60485,12.01782,357.53033,365.41522,32.0000,6.4151
30718,2020-07-11 08:31:58,380.93369,12,10,47.97516,-1.5981,60.37292,772.53174,28.02996,96.0,...,874.83227,0.0,1.42456,12.43378,35.63538,12.01782,358.76013,367.67578,32.0000,6.4215


In [62]:
TIMESTAMP_FIELD = "time"
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = "attack"
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])
VALID_COLUMNS_IN_TRAIN_DATASET

Index(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79'],
      dtype='object')

#### 2. 데이터 전처리

In [63]:
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

In [64]:
TAG_MIN 

C01    357.26651
C02     12.00000
C03     10.00000
C04     47.60895
C05     -2.02130
         ...    
C75     12.00256
C76    233.56122
C77    229.87201
C78     32.00000
C79      4.98040
Length: 79, dtype: float64

- Min-Max Normalization (최소-최대 정규화)
    - (X - MIN) / (MAX-MIN)
    - 모든 feature에 대해 각각의 최소값 0, 최대값 1로, 그리고 다른 값들은 0과 1 사이의 값으로 변환

In [65]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

- ewm 은 지수가중함수
    - 오래된 데이터에 지수감쇠를 적용하여 최근 데이터가 더 큰 영향을 끼지도록 가중치를 주는 함수
    - 추가 메서드로 mean() 을 사용해서 지수가중평균으로 사용

    - 사용법
        - df.ewm(com=None, span=None, halflife=None, alpha=None, min_periods=0, adjust=True, ignore_na=False, axis=0, times=None, method='single')
        - 주로 가중치를 결정하는 요소는 alpha!! 평활계수(감쇠계수) ====>  com / span / halflife를 통해 자동 계산하도록 하거나, alpha를 통해 직접 설정

In [66]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TRAIN_DF

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,0.688950,0.0,0.0,0.238278,0.547276,0.000231,0.485211,0.482296,0.784144,0.790333,...,0.794687,0.000000e+00,0.246867,0.345766,0.340421,0.500000,0.279926,0.273943,1.000000e+00,0.724537
1,0.688107,0.0,0.0,0.237642,0.514459,0.000161,0.507654,0.424017,0.794139,0.847008,...,0.804404,0.000000e+00,0.246867,0.345200,0.346874,0.500000,0.277222,0.260272,1.000000e+00,0.624735
2,0.688031,0.0,0.0,0.237900,0.453823,0.000155,0.497133,0.534275,0.803903,0.843979,...,0.809131,0.000000e+00,0.246356,0.346708,0.347455,0.500000,0.275638,0.263286,1.000000e+00,0.547759
3,0.688858,0.0,0.0,0.237926,0.314854,0.000154,0.497884,0.372084,0.813725,0.705696,...,0.809984,0.000000e+00,0.246306,0.349843,0.341124,0.500000,0.274716,0.261526,1.000000e+00,0.505503
4,0.691449,0.0,0.0,0.237928,0.187876,0.000154,0.532586,0.789952,0.823039,0.485911,...,0.813532,0.000000e+00,0.246811,0.348244,0.346880,0.500000,0.273030,0.256048,1.000000e+00,0.562004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30715,0.405590,0.0,0.0,0.016445,0.118685,0.686615,0.410469,0.515061,0.009266,0.037350,...,0.858302,8.999903e-09,0.288451,0.554037,0.418438,0.500000,0.479118,0.511163,6.860029e-85,0.874857
30716,0.414933,0.0,0.0,0.016445,0.186946,0.668338,0.445844,0.544611,0.008739,0.088345,...,0.859537,8.999903e-10,0.289421,0.556194,0.424821,0.275000,0.482676,0.519640,6.860029e-86,0.910385
30717,0.423387,0.0,0.0,0.016445,0.280758,0.649399,0.366671,0.488527,0.008166,0.191201,...,0.858891,8.999903e-11,0.287988,0.555249,0.419076,0.477500,0.485263,0.519428,6.860029e-87,0.930317
30718,0.429247,0.0,0.0,0.016760,0.303617,0.630393,0.327666,0.600660,0.007587,0.217679,...,0.859981,8.999903e-12,0.288864,0.556894,0.431273,0.497750,0.489857,0.527203,6.860029e-88,0.936054


In [67]:
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

In [68]:
boundary_check(TRAIN_DF)

(False, False, False)

#### 3. 학습 모델 설정 & 데이터 입출력 정의

In [69]:
WINDOW_GIVEN = 59
WINDOW_SIZE= 60

def HaiDataset(timestamps, df, stride=1, attacks=None):
    ts= np.array(timestamps)
    tag_values=np.array(df,dtype=np.float32)
    valid_idxs=[]
    for L in trange(len(ts)-WINDOW_SIZE+1):
        R = L + WINDOW_SIZE - 1
        if dateutil.parser.parse(ts[R]) - dateutil.parser.parse(ts[L]) == timedelta(seconds=WINDOW_SIZE - 1):
            valid_idxs.append(L)
    valid_idxs=np.array(valid_idxs, dtype=np.int32)[::stride]
    n_idxs = len(valid_idxs)
    print("# of valid windows:", n_idxs)
    if attacks is not None:
        attacks = np.array(attacks, dtype=np.float32)
        with_attack = True
    else:
        with_attack = False
        
    
    timestamp, X, y, att = list(), list(), list(), list()
    
    if with_attack:
        for i in valid_idxs:
            last=i+WINDOW_SIZE-1
            seq_time, seq_x, seq_y, seq_attack = ts[last], tag_values[i:i+WINDOW_GIVEN], tag_values[last], attacks[last]
            timestamp.append(seq_time)
            X.append(seq_x)
            y.append(seq_y)
            att.append(seq_attack)
            
        return np.array(timestamp), np.array(X), np.array(y), np.array(att)
    else:
        for i in valid_idxs:
            last=i+WINDOW_SIZE-1
            seq_time, seq_x, seq_y = ts[last], tag_values[i:i+WINDOW_GIVEN], tag_values[last]
            timestamp.append(seq_time)
            X.append(seq_x)
            y.append(seq_y)
        return np.array(timestamp), np.array(X), np.array(y)

In [70]:
ts, X_train, y_train = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)

  0%|          | 0/30661 [00:00<?, ?it/s]

# of valid windows: 30661


In [71]:
HAI_DATASET_TRAIN = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=10)
HAI_DATASET_TRAIN[0]

  0%|          | 0/30661 [00:00<?, ?it/s]

# of valid windows: 3067


array(['2020-07-11 00:00:59', '2020-07-11 00:01:09',
       '2020-07-11 00:01:19', ..., '2020-07-11 08:31:39',
       '2020-07-11 08:31:49', '2020-07-11 08:31:59'], dtype='<U19')

In [72]:
## Validation dataset load

VALIDATION_DF_RAW = dataframe_from_csvs(VALIDATION_DATASET)
VALIDATION_DF_RAW = VALIDATION_DF_RAW[:1440]
VALIDATION_DF_RAW

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C71,C72,C73,C74,C75,C76,C77,C78,C79,attack
0,2020-07-07 15:00:00,402.70947,12.0,10,51.95007,-1.0189,-1.86768,789.76508,28.03162,688,...,0.0,1.34293,10.89290,34.88770,12.26196,380.31683,386.26666,32.59527,5.6330,0
1,2020-07-07 15:00:01,402.81174,12.0,10,51.96533,-1.2637,-1.86768,789.13147,28.02301,648,...,0.0,1.34216,10.80512,34.88770,12.26196,380.02747,386.30286,32.59527,5.4158,0
2,2020-07-07 15:00:02,402.76062,12.0,10,51.96533,-1.5398,-1.86768,785.81653,28.02993,616,...,0.0,1.34369,10.80029,34.88770,12.26196,381.52850,389.73883,32.59527,5.5532,0
3,2020-07-07 15:00:03,402.81174,12.0,10,51.98822,-1.6212,-1.86768,785.42438,28.02993,584,...,0.0,1.34445,10.80579,34.88770,12.26196,382.08911,388.94311,32.59527,5.7833,0
4,2020-07-07 15:00:04,402.91394,12.0,10,51.90429,-1.5631,-1.86768,782.99249,28.02990,552,...,0.0,1.34293,10.81415,34.90295,12.26196,383.44543,389.72082,32.59527,6.0309,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2020-07-07 15:23:55,401.38046,12.0,10,51.14135,-1.2443,-1.87531,784.32019,28.03422,6800,...,0.0,1.34140,8.02892,36.01685,12.26196,330.62067,332.46527,32.59527,5.9823,0
1436,2020-07-07 15:23:56,401.27820,12.0,10,51.14135,-0.9583,-1.87531,786.50073,28.03078,6456,...,0.0,1.34140,8.09584,36.03211,12.26196,330.45795,331.12707,32.59527,6.0808,0
1437,2020-07-07 15:23:57,401.32935,12.0,10,51.14135,-0.7395,-1.87531,789.28980,28.02906,6128,...,0.0,1.34064,8.12336,36.01685,12.26196,330.45795,331.05469,32.59527,5.7884,0
1438,2020-07-07 15:23:58,401.48270,12.0,10,51.14135,-0.8403,-1.87531,788.40533,28.03426,5824,...,0.0,1.34140,8.10850,36.01685,12.26196,330.25897,329.71643,32.59527,5.3944,0


In [73]:
VALIDATION_DF = normalize(VALIDATION_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET])

In [74]:
boundary_check(VALIDATION_DF)

(True, True, False)

In [75]:
ts, X_valid, y_valid, attack = HaiDataset(VALIDATION_DF_RAW[TIMESTAMP_FIELD], VALIDATION_DF, attacks=VALIDATION_DF_RAW[ATTACK_FIELD])

  0%|          | 0/1381 [00:00<?, ?it/s]

# of valid windows: 1381


- EarlyStopping 지정~!
    - 과적합을 방지하기 위해서는 EarlyStopping 이라는 콜백함수를 사용하여 적절한 시점에 학습을 조기 종료
    


- ModelCheckpoint
    - 모델이 학습하면서 정의한 조건을 만족했을 때 Model의 weight 값을 중간 저장

In [76]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_LSTM_model_60seq_TEST.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

#### 4. Training Model

In [77]:
## LSTM Model define

n_features = TRAIN_DF.shape[1]

inputs=Input(shape=(WINDOW_GIVEN , n_features))
first=Bidirectional(LSTM(100, return_sequences=True))(inputs)
second=Bidirectional(LSTM(100, return_sequences=True))(first)
thrid=Bidirectional(LSTM(100))(second)

lstm_out=Dense(n_features)(thrid)

aux_input = Input(shape=(n_features,), name='aux_input')

outputs = keras.layers.add([lstm_out, aux_input])

model_60seq=Model(inputs=[inputs, aux_input], outputs=outputs)

model_60seq.compile(loss='mean_squared_error', optimizer='Adam')

model_60seq.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 59, 79)]     0           []                               
                                                                                                  
 bidirectional_3 (Bidirectional  (None, 59, 200)     144000      ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 bidirectional_4 (Bidirectional  (None, 59, 200)     240800      ['bidirectional_3[0][0]']        
 )                                                                                                
                                                                                            

In [78]:
aux_train=[]

for i in range(len(X_train)):
    aux_train.append(X_train[i][0])
aux_train=np.array(aux_train)

In [79]:
aux_valid=[]
for i in range(len(X_valid)):
    aux_valid.append(X_valid[i][0])
aux_valid=np.array(aux_valid)

In [80]:
hist = model_60seq.fit([X_train, aux_train], y_train, batch_size=512, epochs=32, shuffle=True, 
               callbacks=[es, mc], validation_data=([X_valid,aux_valid],y_valid))

Epoch 1/32


KeyboardInterrupt: 

In [81]:
model_60seq.save('best_LSTM_model_60seq.h5') #keras h5

#### 5. Skip learning

In [82]:
## model load
model_60seq = load_model('best_LSTM_model_60seq.h5')

In [83]:
## Validation set Prediction

y_pred=model_60seq.predict([X_valid,aux_valid])

In [84]:
tmp=[]
for i in range(len(y_pred)):
    tmp.append(abs(y_valid[i]-y_pred[i]))

In [85]:
tmp

[array([0.05050147, 0.10788567, 0.07237996, 0.03742538, 0.19029623,
        0.10945562, 0.16381705, 0.17136168, 0.0472691 , 0.1832028 ,
        0.13826054, 0.07677621, 0.20098108, 0.3378035 , 0.13348645,
        0.1423893 , 0.14752853, 0.05783464, 0.03795845, 0.05382222,
        0.07704217, 0.3384416 , 0.05946273, 0.00328672, 0.14941975,
        0.07324514, 0.26259577, 0.241644  , 0.10193992, 0.06678362,
        0.10051757, 0.23262928, 0.50186586, 0.09651529, 0.09725866,
        0.05270553, 0.04814655, 0.03172931, 0.11518496, 0.17011161,
        0.02425903, 0.10192799, 0.35204318, 0.40993726, 0.00061136,
        0.4835915 , 0.10617191, 0.40076405, 0.12430006, 0.07953614,
        0.03254866, 0.03602917, 0.2326761 , 0.12940788, 0.01196274,
        0.03150922, 0.01737517, 0.08929211, 0.01203741, 0.00544131,
        0.13990417, 0.18395996, 0.24174926, 0.02285499, 0.03516699,
        0.23893425, 0.08605484, 0.16117871, 0.1947747 , 0.03086543,
        0.08389655, 0.24108417, 0.22895366, 0.09

In [86]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

- threshold 이용해 label 값 넣기

In [87]:
def put_labels(distance, threshold):
    xs = np.zeros_like(distance)
    xs[distance > threshold] = 1
    return xs

In [88]:
THRESHOLD = 0.027

LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)
LABELS, LABELS.shape

(array([1., 1., 1., ..., 1., 1., 1.], dtype=float32), (1381,))

In [89]:
ATTACK_LABELS = put_labels(np.array(VALIDATION_DF_RAW[ATTACK_FIELD]), threshold=0.5)
ATTACK_LABELS, ATTACK_LABELS.shape

(array([0, 0, 0, ..., 0, 0, 0]), (1440,))

In [90]:
def fill_blank(check_ts, labels, total_ts):
    def ts_generator():
        for t in total_ts:
            yield dateutil.parser.parse(t)

    def label_generator():
        for t, label in zip(check_ts, labels):
            yield dateutil.parser.parse(t), label

    g_ts = ts_generator()
    g_label = label_generator()
    final_labels = []

    try:
        current = next(g_ts)
        ts_label, label = next(g_label)
        while True:
            if current > ts_label:
                ts_label, label = next(g_label)
                continue
            elif current < ts_label:
                final_labels.append(0)
                current = next(g_ts)
                continue
            final_labels.append(label)
            current = next(g_ts)
            ts_label, label = next(g_label)
    except StopIteration:
        return np.array(final_labels, dtype=np.int8)

In [91]:
%%time
FINAL_LABELS = fill_blank(ts, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS.shape

CPU times: user 133 ms, sys: 1.91 ms, total: 135 ms
Wall time: 131 ms


(1440,)

In [92]:
ATTACK_LABELS.shape[0] == FINAL_LABELS.shape[0]

True

In [93]:
# 성능 평가도구 eTaPR(enhanced Time-series aware Precision and Recall)
TaPR = etapr.evaluate(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)

print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

F1: 0.000 (TaP: 0.000, TaR: 0.000)
# of detected anomalies: 0
Detected anomalies: []


#### 6. Moving Average

In [94]:
## MOVING AVERAGE

seq60_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq60_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq60_10mean.append(ANOMALY_SCORE[idx])

seq60_10mean=np.array(seq60_10mean)
print(seq60_10mean.shape)

(1381,)


In [95]:
THRESHOLD = 0.019

LABELS = put_labels(seq60_10mean, THRESHOLD)
LABELS, LABELS.shape

(array([1., 1., 1., ..., 1., 1., 1.]), (1381,))

In [96]:
%%time
FINAL_LABELS = fill_blank(ts, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS.shape

CPU times: user 131 ms, sys: 574 µs, total: 132 ms
Wall time: 129 ms


(1440,)

In [97]:
ATTACK_LABELS.shape[0] == FINAL_LABELS.shape[0]

True

In [98]:
TaPR = etapr.evaluate(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

F1: 0.000 (TaP: 0.000, TaR: 0.000)
# of detected anomalies: 0
Detected anomalies: []


- test data 위와 같이

In [99]:
############### Test data Start

TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)
TEST_DF_RAW = TEST_DF_RAW[:11960]
TEST_DF_RAW

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,2020-07-09 15:00:00,384.30737,12.0,10,70.35980,-1.6171,-1.79901,774.20752,28.02385,136,...,936.58447,0.0,1.35437,13.97231,35.22338,12.02545,293.51129,283.92651,32.0000,6.5059
1,2020-07-09 15:00:01,384.30737,12.0,10,70.35980,-1.7606,-1.79901,772.58758,28.02730,136,...,940.93317,0.0,1.35437,13.93358,35.20813,12.02545,292.67938,283.36591,32.0000,6.3079
2,2020-07-09 15:00:02,384.20517,12.0,10,70.35980,-1.7606,-1.80664,772.58758,28.02730,136,...,936.58447,0.0,1.35513,13.95248,35.20813,12.02545,291.90179,282.93189,32.0000,6.3079
3,2020-07-09 15:00:03,384.25626,12.0,10,70.35980,-1.7814,-1.79901,777.48810,28.02905,136,...,933.54034,0.0,1.35513,13.89971,35.20813,12.02545,291.59430,282.06378,32.0000,6.1203
4,2020-07-09 15:00:04,384.20517,12.0,10,70.35980,-1.7370,-1.79901,778.42212,28.03169,136,...,944.41223,0.0,1.35437,13.94603,35.20813,12.02545,289.87628,283.67334,32.0000,5.9543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11955,2020-07-09 18:19:15,404.60083,12.0,10,51.98822,-1.5671,-1.86768,778.18188,28.03078,0,...,860.48126,0.0,1.46042,7.87800,35.11657,12.01782,461.85980,459.11096,33.1855,5.7990
11956,2020-07-09 18:19:16,404.75415,12.0,10,51.98059,-1.5189,-1.86768,775.23889,28.02736,0,...,858.74176,0.0,1.46195,7.82746,35.10132,12.01782,462.65552,460.61200,33.1855,5.7826
11957,2020-07-09 18:19:17,404.90750,12.0,10,51.97296,-1.2949,-1.86768,775.77014,28.02649,0,...,855.26282,0.0,1.46118,7.88739,35.11657,12.01782,462.90869,459.59924,33.1855,5.8514
11958,2020-07-09 18:19:18,405.06085,12.0,10,51.97296,-1.1648,-1.86768,772.73779,28.03338,0,...,860.48126,0.0,1.46042,7.82922,35.10132,12.01782,463.12573,462.05872,33.1855,6.0833


In [100]:
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TEST_DF

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,0.491179,0.0,0.0,1.043387,0.292411,0.001001,0.361733,0.385389,0.010417,0.148055,...,0.920815,0.0,0.236630,0.646516,0.241132,0.750000,0.234807,0.207138,0.000000,0.991550
1,0.491179,0.0,0.0,1.043387,0.198036,0.001001,0.327986,0.502287,0.010417,0.143830,...,0.924702,0.0,0.236630,0.644469,0.234688,0.750000,0.231844,0.205185,0.000000,0.874553
2,0.489506,0.0,0.0,1.043387,0.189534,0.000931,0.324946,0.512818,0.010417,0.143449,...,0.921200,0.0,0.237140,0.645274,0.234107,0.750000,0.228834,0.203511,0.000000,0.864013
3,0.490176,0.0,0.0,1.043387,0.175148,0.000994,0.425722,0.572570,0.010417,0.235620,...,0.918159,0.0,0.237191,0.642593,0.234050,0.750000,0.227452,0.200351,0.000000,0.753215
4,0.489408,0.0,0.0,1.043387,0.202620,0.001000,0.455053,0.667098,0.010417,0.368577,...,0.927476,0.0,0.236686,0.644748,0.234044,0.750000,0.221258,0.205586,0.000000,0.645038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11955,0.859228,0.0,0.0,0.200770,0.322986,0.000308,0.445555,0.610518,0.000579,0.649962,...,0.844728,0.0,0.315766,0.292176,0.192764,0.522500,0.894436,0.878798,0.716098,0.535472
11956,0.862246,0.0,0.0,0.200518,0.359406,0.000308,0.391386,0.525644,0.000579,0.709014,...,0.844327,0.0,0.316732,0.289586,0.185235,0.502250,0.897009,0.883660,0.716098,0.522822
11957,0.865054,0.0,0.0,0.200178,0.508891,0.000308,0.396925,0.487972,0.000579,0.863047,...,0.841209,0.0,0.316311,0.292463,0.190862,0.500225,0.898158,0.880653,0.716098,0.561805
11958,0.867842,0.0,0.0,0.200144,0.608547,0.000308,0.334941,0.715327,0.000579,0.853115,...,0.845515,0.0,0.315759,0.289707,0.185045,0.500023,0.899038,0.888835,0.716098,0.701361


In [101]:
boundary_check(TEST_DF)

(True, True, False)

In [102]:
CHECK_TS, X_test, y_test = HaiDataset(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, attacks=None)

  0%|          | 0/11901 [00:00<?, ?it/s]

# of valid windows: 11901


In [103]:
aux_test=[]
for i in range(len(X_test)):
    aux_test.append(X_test[i][0])
aux_test=np.array(aux_test)

In [104]:
## Model Prediction

y_pred=model_60seq.predict([X_test,aux_test])

In [105]:
tmp=[]
for i in range(len(y_test)):
    tmp.append(abs(y_test[i]-y_pred[i]))

In [106]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

In [107]:
# Moving Average

seq60_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq60_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq60_10mean.append(ANOMALY_SCORE[idx])

seq60_10mean=np.array(seq60_10mean)
print(seq60_10mean.shape)

(11901,)


In [108]:
### Threshold Setting

THRESHOLD=0.019

LABELS_60seq = put_labels(seq60_10mean, THRESHOLD)
LABELS_60seq, LABELS_60seq.shape

(array([1., 1., 1., ..., 1., 1., 1.]), (11901,))

In [109]:
submission = pd.read_csv('/home/ubuntu/coding/220906/HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission.loc[CHECK_TS,'attack'] = LABELS_60seq

submission

,time,attack
time,,
2020-07-09 15:00:00,2020-07-09 15:00:00,0
2020-07-09 15:00:01,2020-07-09 15:00:01,0
2020-07-09 15:00:02,2020-07-09 15:00:02,0
2020-07-09 15:00:03,2020-07-09 15:00:03,0
2020-07-09 15:00:04,2020-07-09 15:00:04,0
...,...,...
2020-07-31 12:29:56,2020-07-31 12:29:56,0
2020-07-31 12:29:57,2020-07-31 12:29:57,0
2020-07-31 12:29:58,2020-07-31 12:29:58,0


In [110]:
submission.to_csv('SILab_Moving_Average.csv', index=False)

#### 7. Gray Aare Smoothing

In [111]:
submission = pd.read_csv('/home/ubuntu/coding/220906/HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission['attack_1']=0
submission['anomaly_score']=0

submission.loc[CHECK_TS,'anomaly_score'] = seq60_10mean
submission.loc[CHECK_TS,'attack_1'] = LABELS_60seq

LABELS_60seq=submission['attack_1']
seq60_10mean=submission['anomaly_score']

In [112]:
def Gray_Area(attacks):
    start = []  # start point
    finish = []  # finish point
    c = []  # count
    com = 0
    count = 0
    for i in range(1, len(attacks)):
        if attacks[i - 1] != attacks[i]:
            if com == 0:
                start.append(i)
                count = count + 1
                com = 1
            elif com == 1:
                finish.append(i - 1)
                c.append(count)
                count = 0
                start.append(i)
                count = count + 1
        else:
            count = count + 1

    finish.append(len(attacks) - 1)
    c.append(finish[len(finish) - 1] - start[len(start) - 1] + 1)

    for i in range(0, len(start)):
        if c[i] < 10:
            s = start[i]
            f = finish[i] + 1
            g1 = [1 for i in range(c[i])] # Temp Attack list
            g0 = [0 for i in range(c[i])]  # Temp Normal List
            if attacks[start[i] - 1] == 1:
                attacks[s:f] = g1  # change to attack
            else:
                attacks[s:f] = g0  # change to normal

    return attacks

In [113]:
gray_LABELS_60seq=Gray_Area(LABELS_60seq)

In [114]:
submission = pd.read_csv('/home/ubuntu/coding/220906/HAI 2.0/sample_submission.csv')
submission.index = submission['time']

submission['attack']=gray_LABELS_60seq

submission

,time,attack
time,,
2020-07-09 15:00:00,2020-07-09 15:00:00,0
2020-07-09 15:00:01,2020-07-09 15:00:01,0
2020-07-09 15:00:02,2020-07-09 15:00:02,0
2020-07-09 15:00:03,2020-07-09 15:00:03,0
2020-07-09 15:00:04,2020-07-09 15:00:04,0
...,...,...
2020-07-31 12:29:56,2020-07-31 12:29:56,0
2020-07-31 12:29:57,2020-07-31 12:29:57,0
2020-07-31 12:29:58,2020-07-31 12:29:58,0


In [115]:
submission.to_csv('SILab_Gray_Area_Smoothing.csv', index=False)

#### 8. Attack Point and Finish Point Policy

In [116]:
WINDOW_GIVEN = 9
WINDOW_SIZE= 10

In [117]:
ts, X_train, y_train = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)

  0%|          | 0/30711 [00:00<?, ?it/s]

# of valid windows: 30711


In [118]:
ts, X_valid, y_valid, attack = HaiDataset(VALIDATION_DF_RAW[TIMESTAMP_FIELD], 
                                          VALIDATION_DF, attacks=VALIDATION_DF_RAW[ATTACK_FIELD])

  0%|          | 0/1431 [00:00<?, ?it/s]

# of valid windows: 1431


In [119]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_LSTM_model_10seq_TEST.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [120]:
## LSTM Model define

n_features = TRAIN_DF.shape[1]

inputs=Input(shape=(WINDOW_GIVEN , n_features))
first=Bidirectional(LSTM(100, return_sequences=True))(inputs)
second=Bidirectional(LSTM(100, return_sequences=True))(first)
thrid=Bidirectional(LSTM(100))(second)

lstm_out=Dense(n_features)(thrid)

aux_input = Input(shape=(n_features,), name='aux_input')

outputs = keras.layers.add([lstm_out, aux_input])

model_10seq=Model(inputs=[inputs, aux_input], outputs=outputs)

model_10seq.compile(loss='mean_squared_error', optimizer='Adam')

model_10seq.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 9, 79)]      0           []                               
                                                                                                  
 bidirectional_6 (Bidirectional  (None, 9, 200)      144000      ['input_3[0][0]']                
 )                                                                                                
                                                                                                  
 bidirectional_7 (Bidirectional  (None, 9, 200)      240800      ['bidirectional_6[0][0]']        
 )                                                                                                
                                                                                            

In [121]:
aux_train=[]
for i in range(len(X_train)):
    aux_train.append(X_train[i][0])
aux_train=np.array(aux_train)

In [122]:
aux_valid=[]
for i in range(len(X_valid)):
    aux_valid.append(X_valid[i][0])
aux_valid=np.array(aux_valid)

In [123]:
hist=model_10seq.fit([X_train, aux_train], y_train, batch_size=512, epochs=32, shuffle=True, 
               callbacks=[es, mc], validation_data=([X_valid,aux_valid],y_valid))

Epoch 1/32
60/60 [==============================] - ETA: 0s - loss: 0.0089
Epoch 1: val_loss improved from inf to 0.02123, saving model to best_LSTM_model_10seq_TEST.h5
60/60 [==============================] - 36s 503ms/step - loss: 0.0089 - val_loss: 0.0212
Epoch 2/32
60/60 [==============================] - ETA: 0s - loss: 0.0049
Epoch 2: val_loss did not improve from 0.02123
60/60 [==============================] - 30s 507ms/step - loss: 0.0049 - val_loss: 0.0231
Epoch 3/32
60/60 [==============================] - ETA: 0s - loss: 0.0030
Epoch 3: val_loss did not improve from 0.02123
60/60 [==============================] - 29s 484ms/step - loss: 0.0030 - val_loss: 0.0254
Epoch 4/32
60/60 [==============================] - ETA: 0s - loss: 0.0023
Epoch 4: val_loss did not improve from 0.02123
60/60 [==============================] - 29s 489ms/step - loss: 0.0023 - val_loss: 0.0255
Epoch 5/32
60/60 [==============================] - ETA: 0s - loss: 0.0022
Epoch 5: val_loss did not impr

In [124]:
model_10seq.save('best_LSTM_model_10seq.h5') #keras h5

In [125]:
## model load
model_10seq = load_model('best_LSTM_model_10seq.h5')

In [126]:
CHECK_TS, X_test, y_test = HaiDataset(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, attacks=None)

  0%|          | 0/11951 [00:00<?, ?it/s]

# of valid windows: 11951


In [127]:
aux_test=[]
for i in range(len(X_test)):
    aux_test.append(X_test[i][0])
aux_test=np.array(aux_test)

In [128]:
## Model Prediction

y_pred=model_10seq.predict([X_test,aux_test])

In [129]:
tmp=[]
for i in range(len(y_test)):
    tmp.append(abs(y_test[i]-y_pred[i]))

In [130]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

In [131]:
# Moving Average

seq10_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq10_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq10_10mean.append(ANOMALY_SCORE[idx])

seq10_10mean=np.array(seq10_10mean)
print(seq10_10mean.shape)

(11951,)


In [132]:
### Threshold Setting

THRESHOLD=0.008

LABELS_10seq = put_labels(seq10_10mean, THRESHOLD)

LABELS_10seq, LABELS_10seq.shape

(array([1., 1., 1., ..., 1., 1., 1.]), (11951,))

In [133]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission['attack_1']=0

submission.loc[CHECK_TS,'attack_1'] = LABELS_10seq

LABELS_10seq=submission['attack_1']

In [134]:
gray_LABELS_10seq=Gray_Area(LABELS_10seq)

In [135]:
Queue=[0 for i in range(59)]

Label=[]

for i in range(len(gray_LABELS_60seq)):
    Queue.append(gray_LABELS_60seq[i])
    N=Queue.count(1)
    
    if N>=60:
        if seq60_10mean[i-100:i].max() > 0.1:
            Label.append(gray_LABELS_10seq[i])
        else:
            Label.append(gray_LABELS_60seq[i])
        
    elif N>=1:
        Label.append(gray_LABELS_60seq[i])
    
    else:
        Label.append(gray_LABELS_60seq[i])
    
    del Queue[0]

Label=np.array(Label)

In [136]:
submission = pd.read_csv('/home/ubuntu/coding/220906/HAI 2.0/sample_submission.csv')
submission.index = submission['time']

submission['attack']=Label

submission

,time,attack
time,,
2020-07-09 15:00:00,2020-07-09 15:00:00,0
2020-07-09 15:00:01,2020-07-09 15:00:01,0
2020-07-09 15:00:02,2020-07-09 15:00:02,0
2020-07-09 15:00:03,2020-07-09 15:00:03,0
2020-07-09 15:00:04,2020-07-09 15:00:04,0
...,...,...
2020-07-31 12:29:56,2020-07-31 12:29:56,0
2020-07-31 12:29:57,2020-07-31 12:29:57,0
2020-07-31 12:29:58,2020-07-31 12:29:58,0
